In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import seaborn as sns

In [2]:
sc.set_figure_params(dpi=100)

In [3]:
folders = [os.path.join("outputs", folder) for folder in os.listdir("outputs") if "output" in folder and "segmentation_cell_stats.csv" in os.listdir(os.path.join("outputs", folder))]
folders

['outputs/output-XETG00088__0011287__Region_1__20240206__171802',
 'outputs/output-XETG00088__0011695__Region_1__20240202__104242',
 'outputs/output-XETG00088__0018775__Region_1__20240216__134551',
 'outputs/output-XETG00088__0011284__Region_1__20240206__171802',
 'outputs/output-XETG00088__0011546__Region_1__20240130__161548',
 'outputs/output-XETG00088__0011707__Region_1__20240202__104242',
 'outputs/output-XETG00088__0011216__Region_1__20240130__161548',
 'outputs/output-XETG00088__0011762__Region_1__20240124__111313']

In [4]:
from tqdm import tqdm

In [5]:
if not os.path.exists("xenium_outs"):
    os.mkdir("xenium_outs")

In [6]:
for folder in tqdm(folders):
    baysor_cells = pd.read_csv(os.path.join(folder, "segmentation_cell_stats.csv"), index_col=0)
    baysor_cells = baysor_cells[baysor_cells.area.notna()]
    baysor_cells["x_centroid"] = baysor_cells["x"].tolist()
    baysor_cells["y_centroid"] = baysor_cells["y"].tolist()
    print(baysor_cells.shape)
    print(baysor_cells.n_transcripts.sum(), baysor_cells.n_transcripts.sum()/baysor_cells.shape[0])
    adata = sc.read_10x_mtx(os.path.join(folder, "baysor_mtx"))
    baysor_cells.index = [f"cell_{idx}" for idx in baysor_cells.index]    
    common = list(set(baysor_cells.index)&set(adata.obs.index))
    print(len(common)==baysor_cells.shape[0])
    adata = adata[baysor_cells.index]    
    adata.obs = baysor_cells.copy()
    adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()
    adata.obs["cell_area"] = adata.obs["area"].tolist()    
    filename = folder.split('/')[-1]
    filename = filename.split("__")[1]
    adata.write(f"xenium_outs/{filename}.h5ad")    

  0%|          | 0/8 [00:00<?, ?it/s]

(573764, 14)
40139664 69.95849164464832
True


 12%|█▎        | 1/8 [00:32<03:46, 32.30s/it]

(557057, 14)
40637275 72.94994049082949
True


 25%|██▌       | 2/8 [01:01<03:02, 30.37s/it]

...

 75%|███████▌  | 6/8 [02:30<00:47, 23.87s/it]

(676855, 14)
39813966 58.82200175813136
True


 88%|████████▊ | 7/8 [03:00<00:25, 25.85s/it]

(378441, 14)
19310550 51.026580100993286
True


100%|██████████| 8/8 [03:17<00:00, 24.65s/it]
